<div style="text-align: center;">
<img src="images/stust.png" alt="STUST" class="center" style="width: 900px;"/>
</div>

<hr style="border:4px solid gray"> </hr>

<div style="text-align: center;">    
<br>    
    
# Control Arduino's LED & MQTT with Threads
# MQTT + I2C

</div>

<br>
<hr style="border:4px solid gray"> </hr>


<hr style="border:2px solid orange"> </hr>

# [Use MQTT with Raspberry Pi](http://emqx.io/blog/use-mqtt-with-raspberry-pi)

### Install MQTT client library `paho-mqtt`
> Source: https://www.emqx.io/blog/use-mqtt-with-raspberry-pi

* Use the source code to install
>```bash
git clone https://github.com/eclipse/paho.mqtt.python 
cd paho.mqtt.python 
python3 setup.py install
>```

* Use pip3 to install
>```
pip3 install paho-mqtt
>```

### Free public MQTT broker provided by EMQ X:

>```text
Broker         : broker.emqx.io
TCP Port       : 1883
Websocket Port : 8083
>```

# [paho-mqtt Project Description](https://pypi.org/project/paho-mqtt/)
> Source: https://pypi.org/project/paho-mqtt/

<hr style="border:2px solid orange"> </hr>

# Control Arduino-UNO's LEDs via I2C
# Arduino Sketch

<hr style="border:0.5px solid gray">

```C++
#include <Wire.h>

#define I2C_ADDR 11
#define R 11
#define Y 12
#define G 13

unsigned len;
byte msg[20];
byte cmd = 0;

// --------------------------------
void setup() {
  Wire.begin(I2C_ADDR);
  Serial.begin(9600);
  pinMode(R,OUTPUT); 
  pinMode(Y,OUTPUT); 
  pinMode(G,OUTPUT); 
  Serial.print("MQTT + I2C Homework 01: ");
  Serial.println(I2C_ADDR);
  Serial.println("=============================");
  delay(1000);    
  // register onReceive event services           
  Wire.onReceive(irsReceive);
}

// --------------------------------
void disp_msg() {
    Serial.print("Received data  : ");
    for(int i=0; i<len; i++) {
      Serial.print(msg[i]);
      Serial.print(" ");
    }
    Serial.println();  
    cmd = 0;
}

// --------------------------------
void LED(int pin, int stu) {
  digitalWrite(pin,stu);  
  cmd = 255;
}

// --------------------------------
void loop() {
  switch(cmd) {
    case 0   : break;
    case 1   : LED(R,0); break;
    case 2   : LED(R,1); break;
    case 3   : LED(Y,0); break;
    case 4   : LED(Y,1); break;
    case 5   : LED(G,0); break;
    case 6   : LED(G,1); break;
    case 255 :
    default  : disp_msg();
  }
  delay(0);
}

// --------------------------------
void irsReceive(int nb) {
  len = 0;
  while (Wire.available())
    msg[len++] = Wire.read();
  cmd = msg[0];  
}
```

<hr style="border:2px solid orange"> </hr>

# Control Arduino-UNO's LEDs via I2C

In [2]:
import smbus2 as smbus 
from time import sleep
from random import randrange, random

I2C_ADDR = 11
PORT = 1
I2C = smbus.SMBus(PORT)

# LED-Control command
R0, R1 = 1, 2
Y0, Y1 = 3, 4
G0, G1 = 5, 6

# send LED-Control command to Arduino with parameters
# cmd : commands
# dat : additional parameters
def toArduino(cmd, dat=[]):
    global I2C
    bytes_msg = bytes(dat)
    print(f"{cmd}",end=" ")
    I2C.write_i2c_block_data(I2C_ADDR, cmd, bytes_msg)
    #sleep(0.1)

ModuleNotFoundError: No module named 'smbus2'

In [ ]:
# send turn-off-LED command to Arduino
def offAll():
    print("offAll() : ", end="")
    for cmd in [R0, Y0, G0]:
        toArduino(cmd)
    print()

# send turn-on-LED command to Arduino
def onAll():
    print("onAll() : ", end="")
    for cmd in [R1, Y1, G1]:
        toArduino(cmd)
    print()    

In [ ]:
# C0  : R0, Y0, G
# t   : blink t times
# t0  : off period
# t1  : on period
def blink(C0, t=3, t0=0.2, t1=0.2):
    print(f"blink({C0},{t},{t0},{t1}) : ", end="")
    while t>0:
        t=t-1
        toArduino(C0+1)
        sleep(t1)
        toArduino(C0)
        sleep(t0)
    print()

In [ ]:
R2G= [
    ([Y0, R1],5),
    ([Y1],0.5),
    ([R0, Y0, G1],4),
    ([G0, Y1],0.5),
]

G2R = [
    ([Y0, R0, G1], 4),
    ([G0, Y1], 0.5),
    ([Y0, R1], 4),
    ([Y1], 0.5),    
]

BRY = [
    ([R1, Y1], 0.5),
    ([R0, Y0], 0.5),
]

# send command sequence with delays
def cmds_seq(seq, loop=1):
    print(f"cmds_seq(..., {loop}) : ", end="")
    while loop>0:
        for cmds, td in seq:
            _ = [toArduino(c) for c in cmds]
            print(f"[{td}s] ", end="")
            sleep(td)
        loop-=1
    print()

In [ ]:
cmds_seq(BRY,5)

In [ ]:
# test cases
if __name__=="__main__":
    onAll()
    sleep(2)
    offAll()
    blink(R0,5)
    blink(Y0,5)
    blink(G0,5,0.5,0.2)
    cmds_seq(BRY, 3)
    cmds_seq(G2R, 3)
    offAll()
    sleep(2)
    cmds_seq(R2G, 3)


<hr style="border:2px solid orange"> </hr>

# Control Arduino-UNO's LEDs by MQTT Message

In [ ]:
import smbus2 as smbus 
from time import sleep
from random import randrange, random

I2C_ADDR = 11
PORT = 1
I2C = smbus.SMBus(PORT)

def cmd2Arduino(cmd, par=[]):
    global I2C
    bytes_msg = bytes(par)
    # print(f"{cmd}, {par}")
    I2C.write_i2c_block_data(I2C_ADDR, cmd, bytes_msg)

In [ ]:
import paho.mqtt.client as mqtt
import threading, os
from time import sleep

TOPIC  = "P203-UNO"
BROKER = "broker.emqx.io"
PORT   = 1883
Client = None

# MQTT-Connected callback function 
def on_connect(client, userdata, flags, rc):
    global TOPIC
    print("Connected with result code "+str(rc))

# MQTT message retrieval callback function for the subscribed topics
def on_message(client, userdata, msg):
    topic = msg.topic
    payload = msg.payload.decode('utf-8')
    print(f"TOPIC({topic}) : MSG({payload})")
    cmds = payload.split(',')
    try:
        if cmds[0]=='EXIT':
            client.disconnect()
        elif cmds[0]=='PAR':
            cmd = int(cmds[1])
            par = [int(c) for c in cmds[2:]]
            cmd2Arduino(cmd, par)
        elif cmds[0]=='CMDS':
            for cmd in cmds[1:]:
                cmd2Arduino(int(cmd))
        else:
            for cmd in cmds:
                cmd2Arduino(int(cmd))
    except:
        print("Something wrong!")
            
def connect_mqtt():
    global Client
    # Create MQTT client and connect to the broker    
    Client = mqtt.Client()
    Client.on_connect = on_connect
    Client.on_message = on_message
    # client.username_pw_set("user-name","password")
    Client.connect(BROKER, PORT, keepalive=60)
    # Clear the retained message send by Kittenblock
    Client.publish(topic=TOPIC, payload=None, retain=True)
    # Subscribe interested topic(s)
    Ｃlient.subscribe(TOPIC)

# Main Loop
def main():
    global Client
    Client.loop_forever()

if __name__ == "__main__":
    connect_mqtt()
    main()

Connected with result code 0


<hr style="border:2px solid orange"> </hr>

# MQTT Client with Threading


In [ ]:
import paho.mqtt.client as mqtt
import threading, os
from time import sleep

TOPIC  = f"P203-100"
BROKER = "broker.emqx.io"
PORT   = 1883
Client = None

# MQTT-Connected callback function 
def on_connect(client, userdata, flags, rc):
    global TOPIC
    print("Connected with result code "+str(rc))

# MQTT message retrieval callback function for the subscribed topics
def on_message(client, userdata, msg):
    global TOPIC
    topic = msg.topic
    payload = msg.payload.decode('utf-8')
    print(f"TOPIC({topic}) : MSG({payload})")
    if topic == TOPIC:
        if payload=='EXIT':
            client.disconnect()

def connect_mqtt():
    global Client
    # Create MQTT client and connect to the broker    
    Client = mqtt.Client()
    Client.on_connect = on_connect
    Client.on_message = on_message
    # client.username_pw_set("user-name","password")
    Client.connect(BROKER, PORT, keepalive=60)
    # Clear the retained message send by Kittenblock
    Client.publish(topic=TOPIC, payload=None, retain=True)
    # Subscribe interested topic(s)
    Ｃlient.subscribe(TOPIC)

# Main Loop
def main():
    global LOOP, Client
    Client.loop_forever()
    print("main: Bye!")
    LOOP = False

# Dummy task for thread demonstration
def task01():
    global LOOP
    while LOOP:
        sleep(1)
    print("task01: Bye!")
    
if __name__ == "__main__":
    connect_mqtt()
    LOOP = True
    t = threading.Thread(target = task01)
    t.start()
    main()
    t.join()
    print("All done.") 

<hr style="border:0.5px solid gray"> 

<br><hr style="border:3px solid red"> </hr>
<div style="text-align: left;">         
    
# *Final Project - I2C Traffic-Light Station with MQTT Remote*

</div>
<hr style="border:3px solid red"> </hr>

## Project Description

* This project requires 2 different development boards: `Raspberry Pi` and `Arduino-UNO`.
* Arduino-UNO `I2C Traffic-Light Station` handles the following commands:
    * *LED SET 1: (time unit : 0.1sec, e.g. td-p=10 => 1sec)*
        * `1`  : RED Led off
        * `2`  : RED Led on
        * `3`  : YELLOW Led off
        * `4`  : YELLOW Led on
        * `5`  : GREEN Led off
        * `6`  : GREEN Led on
        * `11, [td-p]` : RED Led blink (period `td-p`, 50% duty-cycle) 
        * `12, [td-p]` : YELLOW Led blink (period `td-p`, 50% duty-cycle)
        * `21, [td-r, td-ry, td-g, td-y]` : RED-to-GREEN sequence
            * R(`td-r`) -> R+Y(`td-ry`) -> G(`td-g`) -> Y(`td-y`)
        * `22, [td-r, td-ry, td-g, td-y]` : GREEN-to-RED sequence
            * G(`td-g`) -> Y(`td-y`) -> R(`td-r`) -> R+Y(`td-ry`) 

    * *LED SET 2: (refer to LED SET 1, all `command code + 100`)*
        * `101`  : RED Led off
        * `102`  : RED Led on
        * ...

* Raspberry Pi `MQTT Remote Control` handles the following tasks:
    * Perform self-test at startup
    * Provide the minimum command set corresponding to your I2C Traffic-Light Station
    * MQTT command format:
        * simple command: `1`, `2`, `3`, `101`, `102`, ...
        * command with paramaters: `11/10`, `21/50/5/60/5`, ...

## Grading rules:
* `50 points`  : Complete command 1 ~ 6 of LED SET 1 
* `60 points`  : Complete command 1 ~ 6 of LED SET 1 and 2 
* `70 points`  : Complete command 1 ~ 12 of LED SET 1 
* `80 points`  : Complete command 1 ~ 12 of LED SET 1 and 2 
* `90 points`  : Complete all commands of LED SET 1 
* `100 points` : Complete all commands of LED SET 1 and 2 
* *You must complete all required programs (C/C++ and Python) for I2C Traffic-Light Station and MQTT Remote Control to get full points.*

<div style="text-align: left;">
<img src="images/traffic-light.png" alt="Break" class="center" style="width: 300px;"/>
</div>

<br><hr style="border:3px solid red"> </hr>
<div style="text-align: left;">         
    
# *Final Project - I2C Enabled LCD Device with MQTT Remote*

</div>
<hr style="border:3px solid red"> </hr>

## Project Description

<div style="text-align: left;">
<img src="images/i2c-LCD.png" alt="Break" class="center" style="width: 500px;"/>
</div>

<hr style="border:2px solid orange"> </hr>
<br>

<div style="text-align: left;">
<img src="images/break-yang-tr.png" alt="Break" class="center" style="width: 600px;"/>
</div>
